### Milestone 3

EMILY - Free Saturday/Sunday (Possibly free late tonight)

Sebastian - Free late tonight (maybe) and Sunday

JOEY - Free Saturday/Sunday

RISAB - Saturday/Sunday-ish? (busy this weekend, free on mornings)

JUSTIN - Free Saturday/Sunday

CHRISTINE - More free Saturday/Sunday

Steps:
1. Save our encoded data into a CSV and then ==> open it here **(by end of Friday/Saturday noon)**
- Use Milestone 2 data (https://colab.research.google.com/drive/1bjhNxm6oj0MIGyFm1_xxBMnk-qcns9mw?usp=sharing#scrollTo=g1pxUtqV2zIP)
- DONE: EMILY

2. Train your first model **(Saturday/Sunday morning)**
- Get the model from HW2 bean clasifier
- Try to pass it in with our loaded data (see below)
   - Use more than 5_000 data points to prevent overfitting
- OPTIONAL: Hyperparameter tuning
- DONE: JOEY

3. Evaluate your model compare training vs test error
- Again, copy paste from HW2
- Accuracy, precision, recall (try implementing `sklearn.metrics.multilabel_confusion_matrix`) per class

4. Where does your model fit in the fitting graph. (I.e., overfitting, underfitting, etc.)
**It's underfitting**. It's trying to predict the same class every single time.

5. What are the next 2 models you are thinking of and why?
**- Everyone: think of 1 possible model idea to use?**

6. Update your readme with this info added to the readme with links to the jupyter notebook!
- EMILY: Explain why we aren't using images/image input in our model (isn't indicative of $$ necessarily)

7. Conclusion section: What is the conclusion of your 1st model? What can be done to possibly improve it? **(Sunday afternoon meeting)**
- More layers?

Please make sure preprocessing is complete and your first model has been trained, and predictions for train, val and test are done and analyzed.

#Data Preprocessing Imports

In [ ]:
#Add imports here as needed
import pandas as pd
import numpy as np
import json
import urllib
import requests
import gzip
import ast
from collections import Counter
import re

#Sentiment analysis
#!pip install vaderSentiment
#!pip install install googletrans==3.1.0a0
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Geographic object imports
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

# Load Preprocessed Data in Database (Run this ONCE to get the CSV formatted and loaded)

### Loading Place Data
- Filter by prices are not None; standardize prices by $; one-hot encode
- Get geographical location

Load data:

In [ ]:
data_link = 'https://datarepo.eng.ucsd.edu/mcauley_group/data/googlelocal/places.clean.json.gz'

In [ ]:
def load_place(line, place_list):
  #Read a line, remove parsing characters like u"
  place = line.decode('utf-8').strip().replace('u"', '"')

  #Place is now a dictionary
  place = ast.literal_eval(place)

  if (place['price'] is not None) and (place['price'] != '         '):
    #Remove unnecessary keys (hours, phone)
    place.pop('hours', None)
    place.pop('phone', None)

    #Append place to the list
    place_list.append(place)

In [ ]:
place_list = [] #Store in empty list

response = urllib.request.urlopen(data_link) #Open the file at the link

with gzip.open(response, 'rb') as f:
  for line in f:
    load_place(line, place_list=place_list)

#Time to load: 5m 41s
#You may have to restart your runtime to get 5m runtime! If it's 8m>, something is wrong -- you should try again. Make sure your RAM graph looks accurate/fine.

In [ ]:
places = pd.DataFrame(place_list) #Converting to dataframe #Time = 9s

Standardize price from foreign currency to USD

In [ ]:
def convert_to_usd(symbol):
  conversion = { '$': 1, '€': 1.08, '£': 1.26, '₴': 0.027, '₩':0.00075, '฿':0.028, 'R':0.052, '₱':0.01788}
  if len(symbol) > 0 and symbol[0] in conversion:
    return 10 * len(symbol) * conversion[symbol[0]]
  else:
    return None

def convert_to_symbol(dollars):
  # $ = Inexpensive, usually $10 and under
  # $$ = Moderately expensive, usually between $10-$25
  # $$$ = Expensive, usually between $25-$45
  # $$$$ = Very Expensive, usually $50 and up'
  if dollars < 10:
    return '$'
  elif dollars < 25:
    return '$$'
  elif dollars < 45:
    return '$$$'
  return '$$$$'

In [ ]:
places['price'] = places['price'].apply(convert_to_usd) #Convert into USD
places['price'] = places['price'].apply(convert_to_symbol) #Convert into $, $$, $$$, etc.

In [ ]:
places['price'].value_counts()

$$      209259
$$$     192061
$$$$      2543
$          821
Name: price, dtype: int64

One-Hot Encode Price:

In [ ]:
places = pd.get_dummies(places, columns=['price'], prefix='Class')

Get geographical objects and save to `places`

In [ ]:
df_geo = pd.DataFrame(places[['gPlusPlaceId', 'address']])
df_geo = df_geo.assign(latitude = places[places['gps'].notna()]['gps'].apply(lambda x: x[0]))
df_geo = df_geo.assign(longitude = places[places['gps'].notna()]['gps'].apply(lambda x: x[1]))

#Filter for only not-null
df_geo = df_geo[df_geo['latitude'].notna()]

#Get location kmeans cluster
## can adjust k based on needs
kmeans = KMeans(n_clusters = 200, random_state=42)
df_geo['location_cluster'] = kmeans.fit_predict(df_geo[['latitude', 'longitude']])
#Runtime: 1m34s

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
#Add location_cluster to the places database
places['location_cluster'] = pd.Series(df_geo.location_cluster).astype(int)

In [ ]:
places.head()

,name,address,closed,gPlusPlaceId,gps,Class_$,Class_$$,Class_$$$,Class_$$$$,location_cluster
0,T C's Referee Sports Bar,"[5322 W 26th St, Sioux Falls, SD 57106]",False,100327153115986850675,"[43.529494, -96.792244]",0,1,0,0,0.0
1,Old Chicago,"[17960 NW Evergreen Pkwy, Beaverton, OR 97006]",False,118222137795476771294,"[45.535176, -122.862242]",0,1,0,0,0.0
2,China Cottage,"[3718 Wilmington Pike, Dayton, OH 45429]",False,106432060150136868000,"[39.692899, -84.136173]",0,1,0,0,0.0
3,Smokey Mountain Wings,"[3607 Outdoor Sportsman Pl, Kodak, TN 37764]",False,100184392614713668281,"[35.98598, -83.610598]",0,1,0,0,0.0
4,Sabatinos Italian Kitchen,"[242 Massachusetts Ave, Arlington, MA 02474]",False,110300304875024740707,"[42.406904, -71.143994]",0,0,1,0,0.0


### Loading Review Data
- Group by category
- Get rid of non-restaurant google locations
- Translate reviews

Load data:

In [ ]:
data_link = 'https://datarepo.eng.ucsd.edu/mcauley_group/data/googlelocal/reviews.clean.json.gz'

In [ ]:
def load_review(line, review_list):
  #Read a line, remove parsing characters like /n, 'b, u"
  review = line.decode('utf-8').strip().replace("u'", "'")

  #Place is now a dictionary
  review = ast.literal_eval(review)

  if review['gPlusPlaceId'] is not None:
    #Remove unnecessary keys (hours, phone)
    review.pop('reviewerName', None)
    review.pop('phone', None)
    review.pop('gPlusUserId')

    #Append place to the list
    review_list.append(review)

In [ ]:
review_list = [] #Store in empty list

response = urllib.request.urlopen(data_link) #Open the file at the link

with gzip.open(response, 'rb') as f:
  #We want to get 1,000 reviews to start
  for i in range(5000):
    line = f.readline()
    load_review(line, review_list=review_list)

#Time to load data: 1s

In [ ]:
reviews = pd.DataFrame(review_list)

In [ ]:
reviews.head()

,rating,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime
0,3.0,Chất lượng tạm ổn,[Giải Trí - Café],108103314380004200232,1.372687e+09,"Jul 1, 2013"
1,5.0,Wc si temiz duzenli..,[Turkish Cuisine],102194128241608748649,1.342871e+09,"Jul 21, 2012"
2,5.0,何回も私は予定に休みがセルバに行ったので覚えて見て、分かります❗,"[Fishing, Pond Fish Supplier, Seafood Market]",101409858828175402384,1.390654e+09,"Jan 25, 2014"
3,5.0,今度は予定に休みが登米市に行きたい❗☀😅🌌 楽しいに日帰りに登米の見学の観光(*^)(*^-...,[Museum],101477177500158511502,1.389188e+09,"Jan 8, 2014"
4,4.0,気仙沼警察署に移転中に絆 👮🐎☺🙋🚓頑張ろう❗,[Police],106994170641063333085,1.390486e+09,"Jan 23, 2014"


Clean categories and pick the most important one

In [ ]:
from googletrans import Translator

In [ ]:
#Clean categories/translate them
def trans(cats):
  translator=Translator()
  if cats is not None:
    return translator.translate(cats[0]).text
  else:
    return None

In [ ]:
#Apply translation to the first word in the categories column
reviews['top_category'] = reviews['categories'].apply(trans)
#Runtime: 8 minutes

Pick which categories are 'relevant'

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300') #Download Google API to compute word similarity to
#Download time: 11 minutes

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
def similar(category, threshold=0.5):
  sims = []
  if category is None:
    return False

  for word in category.split():
    if word in wv:
      sims.append(wv.similarity(word, 'restaurant'))
    else:
      sims.append(0)

  return max(sims) > threshold

In [ ]:
#Apply threshhold to reviews
relevant_category = reviews['top_category'].apply(similar)
reviews['relevant_cat'] = relevant_category

In [ ]:
#Drop non-relevant places/review tags
indexRel = reviews[reviews.relevant_cat == False].index
reviews.drop(indexRel, inplace=True)

Translate reviews:

In [ ]:
from googletrans import Translator

In [ ]:
#Check language function
def check(word):
  translator=Translator()
  result = translator.translate(word)
  return [result.src, result.text]

In [ ]:
reviewLang = reviews.reviewText.apply(check)
#This took 3m 45s to run for 1943 samples (pre-filtered by category)
#This took 9m 15s to run for 5000 samples
#This took 19m 33s to run for 10,000 samples

In [ ]:
translated = pd.DataFrame(reviewLang)
translated = pd.DataFrame(translated['reviewText'].to_list(), columns = ['language', 'translated']) #Split our [src, text] into two columns
#translated.head()

In [ ]:
#Added columns to our main dataframe
reviews['language'] = translated['language']
reviews['translated'] = translated['translated']

In [ ]:
#Drop the un-translated column
reviews.drop('reviewText', axis=1, inplace=True)

### Use TF-IDF score to find most unique "defining" words of restaurant vibe

In [ ]:
#Let's find the most unique/defining word in the group using TF-IDF:
from sklearn.feature_extraction.text import TfidfVectorizer

cats_comp = list(reviews.top_category) #Train on the top_category for every relevant input

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cats_comp)

tfidf_dict = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))

#tfidf_dict

In [ ]:
def def_cat(category):
  words = {}

  category = category.lower().replace("restaurant", "") #Had to edit filtering requirements because too many restaurants were just categorized as "restaurant"

  if category is None:
    return None

  for word in category.split():
    if word in tfidf_dict:
      words[word] = tfidf_dict[word]

  if len(words) > 0:
    sorted_words = dict(sorted(words.items(), key=lambda item: item[1]))
    return list(sorted_words.keys())[-1]
  else:
    return "restaurant" #if the only category word is "restaurant" we need a catchall

In [ ]:
#Get the defining category
kw = reviews.top_category.apply(def_cat)

In [ ]:
# Compute similarity scores between words
similarity_matrix = []
for word1 in list(kw): #For each word in kw
  similarity_scores = []
  for word2 in list(kw): #Compare to the other words in kw (kw = keyword in category)
    if word2 in wv and word1 in wv:
      similarity_scores.append(wv.similarity(word1, word2)) #Return the similarity score if kw is in wv training library
    else:
      similarity_scores.append(0) #Return a similarity of 0 if can't find it
  similarity_matrix.append(similarity_scores)

#Runtime should be about 1m

In [ ]:
from sklearn.cluster import KMeans

# Perform K-means clustering
num_clusters = 5  # I want 5 clusters for now
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(similarity_matrix)
cluster_labels = kmeans.labels_

#Runtime: 5s

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
#Assign each keyword in kw to a cluster
word_clusters = {}
cat_list = []
for i, word in enumerate(list(kw)):
  cluster_id = cluster_labels[i] #Get the cluster id = what kmeans grouped as in prev cell
  cat_list.append(cluster_id)
  if cluster_id not in word_clusters: #If this is a new cluster #, create a list to start storing keywords
      word_clusters[cluster_id] = []
  word_clusters[cluster_id].append(word) #Add the keyword to word_clusters[number]

In [ ]:
reviews['cat_cluster'] = pd.Series(cat_list)

### Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def sentiment_analysis_and_charged_words(review):
  if type(review) == str and review is not None:
    sentiment = analyzer.polarity_scores(review)
    charged_words = [word for word in review.split() if analyzer.polarity_scores(word)['compound'] != 0]
    return sentiment, charged_words
  else:
    return None, None

def sentiment_analysis_and_charged_words_2(reviews):
    rating_scores = []
    all_words = set()
    charged_words = []
    price_related_keywords = [
        'price', 'cost', 'costly', 'expensive', 'cheap', 'cheapest', 'value', 'best-value', 'money', "money's",
        'worth', 'pricing', 'affordable', 'budget', 'economical', 'budget-friendly',
        'pricey', 'pricy', 'economic', 'bargain', 'deal', 'discount', 'charge', 'fee', 'expense',
        'worth-it', 'sale', 'sales', 'purchase', 'transaction', 'deals',
        'payment', 'payments', 'costly', 'inexpensive', 'cost-effective', 'effective',
        'overpriced', 'underpriced', 'overprice', 'underprice', 'overprices', 'underprices', 'cheaper', 'cheapest', 'pricier', 'priciest',
        'refund', 'income', 'revenue', 'premium', 'garbage', 'inexpensive', 'extravagant', 'boujee', 'rip-off'
        "an arm and a leg", "break the bank", "dent in your wallet", "dent in the wallet", "arm and leg",
         "expensive proposition", "big bucks", "high price", "high cost", "high-price", "high-cost",
        "top dollar", "top-dollar", "cost a pretty penny", "pretty penny", "cost a fortune"
    ]
    if reviews is not None and type(reviews) == str:
      review = reviews.lower()
      for word in price_related_keywords:
        if word in review.split():
          charged_words.append(word)

    return list(set(charged_words))


reviews['sentiment'], reviews['charged_words'] = zip(*reviews['translated'].apply(lambda x: sentiment_analysis_and_charged_words(x)))
reviews['price_words'] = reviews['translated'].apply(sentiment_analysis_and_charged_words_2)

## Merge tables together

In [ ]:
#Left join since we want only restaurants associated with the reviews
df = places.merge(reviews, how='inner', on='gPlusPlaceId')

#Time to merge: 0s

## Cleanup/Standardization

In [ ]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
df['unixReviewTime'] = pd.to_datetime(df['unixReviewTime'], unit='s')

In [ ]:
reviews.columns

Index(['rating', 'categories', 'gPlusPlaceId', 'unixReviewTime', 'reviewTime',
       'top_category', 'relevant_cat', 'language', 'translated', 'cat_cluster',
       'sentiment', 'charged_words', 'price_words'],
      dtype='object')

In [ ]:
#make all review text data an empty string
#take care of this case in data analysis
df['translated'].fillna('', inplace=True)

In [ ]:
df.columns

Index(['name', 'address', 'closed', 'gPlusPlaceId', 'gps', 'Class_$',
       'Class_$$', 'Class_$$$', 'Class_$$$$', 'location_cluster', 'rating',
       'categories', 'unixReviewTime', 'reviewTime', 'top_category',
       'relevant_cat', 'language', 'translated', 'cat_cluster', 'sentiment',
       'charged_words', 'price_words'],
      dtype='object')

In [ ]:
#fill out this information with unknowns
#assume closed is false --> significant amount of data shows that closed is usually false
#can use essentially the mode in this case
df['name'].fillna('Unknown', inplace=True)
df['address'].fillna('Unknown', inplace=True)
df['closed'].fillna(False, inplace=True)
df['gps'].fillna('Unknown', inplace=True)

In [ ]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
df['unixReviewTime'] = pd.to_datetime(df['unixReviewTime'])

In [ ]:
#imputing missing values for time using the mean
#may not be most effective method
#making an assumption that a majority of these reviews were made at the same time which could skew data when training
df['unixReviewTime'].fillna(df['unixReviewTime'].mean(), inplace=True)
df['reviewTime'].fillna(df['reviewTime'].mean(), inplace=True)

In [ ]:
#Get the negative sentiment value
df['sentiment'] = df['sentiment'].apply(lambda x: None if (type(x) is not str) else int(float(x[8:11])))

In [ ]:
language_ohe = pd.get_dummies(df['language'], prefix='Class') #places = pd.get_dummies(places, columns=['price'], prefix='Class')
df = df.join(language_ohe)

In [ ]:
df.drop(['sentiment', 'language'], axis=1, inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

## Save to CSV

In [ ]:
df.to_csv('proc.csv')

# Creating the Model

### Imports

In [ ]:
#pip install scikeras

In [ ]:
#pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import RepeatedKFold, cross_validate
from scikeras.wrappers import KerasClassifier

### Loading the data

In [ ]:
df = pd.read_csv('proc.csv', index_col=0)
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))] #Copied off stackoverflow here: https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe

In [ ]:
df['cat_cluster'] = df['cat_cluster'].fillna(0) #Fill NAs with 0s (should be in above section but i forgot :/)
df['location_cluster'] = df['location_cluster'].fillna(0)

In [ ]:
str_cols = ['name', 'address', 'gps', 'top_category', 'relevant_cat', 'translated', 'charged_words', 'price_words', 'categories', 'unixReviewTime', 'reviewTime']
df_num = df[df.columns.drop(str_cols)]

In [ ]:
df_num

,closed,gPlusPlaceId,location_cluster,rating,cat_cluster,sentiment,Class_ca,Class_da,Class_de,Class_en,...,Class_fr,Class_hi,Class_it,Class_ms,Class_nl,Class_pt,Class_tr,Class_$$,Class_$$$,Class_$$$$
0,False,115757957627721988675,0.0,5.0,4.0,None,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,False,116555416797255000560,0.0,4.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,True,114781865961627441828,0.0,5.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,False,114735143729299990529,0.0,5.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,False,105589668159024738610,0.0,5.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,False,104180837589620084282,0.0,4.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
883,False,110513581755637073383,0.0,3.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
884,False,100311016010349448338,0.0,3.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
885,False,101569325572579174871,0.0,5.0,0.0,None,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
#Get X, Y data
X = df_num.drop(['Class_$$', 'Class_$$$', 'Class_$$$$'], axis=1).astype(float)
y = df_num[['Class_$$', 'Class_$$$', 'Class_$$$$']].astype(float)

display(X.shape)
display(y.shape)

(887, 18)

(887, 3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
display(X_train.shape)
display(y_train.shape)

(709, 18)

(709, 3)

In [ ]:
df_num.columns

Index(['closed', 'gPlusPlaceId', 'location_cluster', 'rating', 'cat_cluster',
       'sentiment', 'Class_ca', 'Class_da', 'Class_de', 'Class_en', 'Class_es',
       'Class_fr', 'Class_hi', 'Class_it', 'Class_ms', 'Class_nl', 'Class_pt',
       'Class_tr', 'Class_$$', 'Class_$$$', 'Class_$$$$'],
      dtype='object')

In [ ]:
X_test[X_test['sentiment'].isna() == False]

,closed,gPlusPlaceId,location_cluster,rating,cat_cluster,sentiment,Class_ca,Class_da,Class_de,Class_en,Class_es,Class_fr,Class_hi,Class_it,Class_ms,Class_nl,Class_pt,Class_tr


## Initializing the Model

In [ ]:
X_test = X_test[['closed', 'location_cluster', 'rating', 'cat_cluster', 'Class_ca', 'Class_da', 'Class_de', 'Class_en', 'Class_es',
       'Class_fr', 'Class_hi', 'Class_it', 'Class_ms', 'Class_nl', 'Class_pt',
       'Class_tr']]
X_train = X_train[['closed', 'location_cluster', 'rating', 'cat_cluster', 'Class_ca', 'Class_da', 'Class_de', 'Class_en', 'Class_es',
       'Class_fr', 'Class_hi', 'Class_it', 'Class_ms', 'Class_nl', 'Class_pt',
       'Class_tr']]

In [ ]:
from kerastuner.tuners import RandomSearch
from keras.layers import Input
from keras.models import Model
from sklearn.metrics import accuracy_score


def build_model(hp):
    input_layer = Input(shape=16)
    x = input_layer

    for i in range(4):
        units = hp.Int(f'units_{i}', min_value=8, max_value=18, step=4)
        activation = hp.Choice(f'activation_{i}', ['relu', 'sigmoid', 'tanh'])
        x = Dense(units=units, activation=activation)(x)
    output = Dense(units=4, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output)

    lr = hp.Float("learning_rate", min_value=0.0001, max_value=0.3, sampling="log")
    epochs = hp.Int('epochs', min_value=50, max_value=200, step=50)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss='binary_focal_crossentropy', metrics=['accuracy'])
    return model

#X_train, X_test, y_train, y_test = train_test_split(df_normalized, classes_encoded, test_size=0.1, random_state=1)

tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=2,
    directory='idk_dir',
    project_name='hyperparams_optimization')

tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

best_hyperparameters = tuner.get_best_hyperparameters()[0]

best_model = tuner.hypermodel.build(best_hyperparameters)

best_model.fit(X_train, y_train, epochs=best_hyperparameters['epochs'])

y_pred_probabilities = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)
y_true_expanded = np.argmax(y_test.values, axis=1)
test_accuracy = accuracy_score(y_true_expanded, y_pred)

print("Test Accuracy:", test_accuracy)
print("Optimized Hyperparameters\nNumber of Units in Each Hidden Layer:", [best_hyperparameters[f'units_{i}'] for i in range(4)])
print("Activation Function for Each Hidden Layer:", [best_hyperparameters[f'activation_{i}'] for i in range(4)])
print(F"Learning Rate: {best_hyperparameters['learning_rate']}\nEpochs: {best_hyperparameters['epochs']}")

Reloading Tuner from idk_dir/hyperparams_optimization/tuner0.json
Epoch 1/200
23/23 [==============================] - 2s 4ms/step - loss: 0.1785 - accuracy: 0.0000e+00
Epoch 2/200
23/23 [==============================] - 0s 4ms/step - loss: 0.1654 - accuracy: 0.0014
Epoch 3/200
23/23 [==============================] - 0s 4ms/step - loss: 0.1541 - accuracy: 0.0113
Epoch 4/200
23/23 [==============================] - 0s 4ms/step - loss: 0.1443 - accuracy: 0.2496
Epoch 5/200
23/23 [==============================] - 0s 5ms/step - loss: 0.1357 - accuracy: 0.6460
Epoch 6/200
23/23 [==============================] - 0s 4ms/step - loss: 0.1282 - accuracy: 0.6460
Epoch 7/200
23/23 [==============================] - 0s 6ms/step - loss: 0.1218 - accuracy: 0.6460
Epoch 8/200
23/23 [==============================] - 0s 4ms/step - loss: 0.1161 - accuracy: 0.6460
Epoch 9/200
23/23 [==============================] - 0s 6ms/step - loss: 0.1111 - accuracy: 0.6460
Epoch 10/200
23/23 [===================

In [ ]:
def build_model(hp):
    input_layer = Input(shape=16)
    x = input_layer

    for i in range(4):
        units = hp.Int(f'units_{i}', min_value=8, max_value=18, step=4)
        activation = hp.Choice(f'activation_{i}', ['relu', 'sigmoid', 'tanh'])
        x = Dense(units=units, activation=activation)(x)
    output = Dense(units=3, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output)

    lr = hp.Float("learning_rate", min_value=0.0001, max_value=0.3, sampling="log")
    epochs = hp.Int('epochs', min_value=50, max_value=200, step=50)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss='binary_focal_crossentropy', metrics=['accuracy'])
    return model

#X_train, X_test, y_train, y_test = train_test_split(df_normalized, classes_encoded, test_size=0.1, random_state=1)

tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=2,
    directory='idk_dir',
    project_name='hyperparams_optimization')

tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

best_hyperparameters = tuner.get_best_hyperparameters()[0]

best_model = tuner.hypermodel.build(best_hyperparameters)

best_model.fit(X_train, y_train, epochs=best_hyperparameters['epochs'])

y_pred_probabilities = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)
y_true_expanded = np.argmax(y_test.values, axis=1)
test_accuracy = accuracy_score(y_true_expanded, y_pred)

print("Test Accuracy:", test_accuracy)
print("Optimized Hyperparameters\nNumber of Units in Each Hidden Layer:", [best_hyperparameters[f'units_{i}'] for i in range(4)])
print("Activation Function for Each Hidden Layer:", [best_hyperparameters[f'activation_{i}'] for i in range(4)])
print(F"Learning Rate: {best_hyperparameters['learning_rate']}\nEpochs: {best_hyperparameters['epochs']}")

from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_true_expanded, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Reloading Tuner from idk_dir/hyperparams_optimization/tuner0.json
Epoch 1/200
23/23 [==============================] - 3s 3ms/step - loss: 0.1611 - accuracy: 0.6460
Epoch 2/200
23/23 [==============================] - 0s 3ms/step - loss: 0.1492 - accuracy: 0.6460
Epoch 3/200
23/23 [==============================] - 0s 3ms/step - loss: 0.1407 - accuracy: 0.6460
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1341 - accuracy: 0.6460
Epoch 5/200
23/23 [==============================] - 0s 3ms/step - loss: 0.1283 - accuracy: 0.6460
Epoch 6/200
23/23 [==============================] - 0s 3ms/step - loss: 0.1237 - accuracy: 0.6460
Epoch 7/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1199 - accuracy: 0.6460
Epoch 8/200
23/23 [==============================] - 0s 3ms/step - loss: 0.1170 - accuracy: 0.6460
Epoch 9/200
23/23 [==============================] - 0s 3ms/step - loss: 0.1149 - accuracy: 0.6460
Epoch 10/200
23/23 [=======================